# dataset compelxity

In [ ]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from radon.complexity import cc_visit
from radon.metrics import h_visit, mi_visit
import re
import warnings
from math import pi
from scipy import stats

warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('deep')
task_order = ['Vulnerability Detection', 'Code Search', 'Code Translation', 'Clone Detection']
task_colors = sns.color_palette('deep', 4)

def calculate_ast_depth(code):
    code = '' if code is None else str(code)
    if not code.strip():
        return 0.0
    try:
        tree = ast.parse(code)
        def get_depth(node, depth=0):
            depths = [depth]
            for child in ast.iter_child_nodes(node):
                depths.append(get_depth(child, depth + 1))
            return max(depths)
        return float(get_depth(tree))
    except:
        return float(max(1, code.count('\n') + 1))

def calculate_cyclomatic_complexity(code):
    code = '' if code is None else str(code)
    if not code.strip():
        return 1.0
    try:
        results = cc_visit(code)
        if results:
            return float(sum(item.complexity for item in results) / len(results))
    except:
        pass
    decisions = len(re.findall(r'\b(if|elif|else|for|while|case|catch|switch|and|or)\b', code, re.I))
    ternary = len(re.findall(r'\?', code))
    logical_ops = len(re.findall(r'(&&|\|\|)', code))
    return float(max(1, 1 + decisions + ternary + logical_ops))

def calculate_halstead_metrics(code):
    code = '' if code is None else str(code)
    if not code.strip():
        return {'volume': 0.0, 'difficulty': 0.0, 'effort': 0.0}
    try:
        result = h_visit(code)
        return {
            'volume': float(result.total.volume or 0),
            'difficulty': float(result.total.difficulty or 0),
            'effort': float(result.total.effort or 0)
        }
    except:
        ops = re.findall(r'[\+\-\*\/\=\<\>\!\&\|\^%~]', code)
        ids = re.findall(r'\b[A-Za-z_]\w*\b', code)
        n1 = len(set(ops)) or 1
        n2 = len(set(ids)) or 1
        N1 = len(ops) or 1
        N2 = len(ids) or 1
        n = n1 + n2
        N = N1 + N2
        vol = N * np.log2(n) if n > 1 else N
        diff = (n1 / 2.0) * (N2 / n2) if n2 > 0 else 1.0
        return {'volume': float(vol), 'difficulty': float(diff), 'effort': float(vol * diff)}

def count_operators_operands(code):
    code = '' if code is None else str(code)
    ops = len(re.findall(r'[\+\-\*\/\=\<\>\!\&\|\^%~]', code))
    words = re.findall(r'\b\w+\b', code)
    keywords = {'if','elif','else','for','while','return','def','class','import','from','try','except','finally','with','as','lambda','yield','in','not','and','or'}
    operands = len([w for w in words if w not in keywords])
    return float(ops), float(operands)

def calculate_maintainability_index(code):
    code = '' if code is None else str(code)
    if not code.strip():
        return 100.0
    try:
        return float(mi_visit(code, True))
    except:
        return 100.0

def analyze_dataframe(df, code_column, keep_label=False, label_column=None):
    rows = []
    for _, row in df.iterrows():
        code = row.get(code_column, '')
        if pd.isna(code) or not str(code).strip():
            continue
        ast_depth = calculate_ast_depth(code)
        cyclomatic = calculate_cyclomatic_complexity(code)
        halstead = calculate_halstead_metrics(code)
        operators, operands = count_operators_operands(code)
        maintainability = calculate_maintainability_index(code)
        loc = len(str(code).split('\n'))
        entry = {
            'ast_depth': ast_depth,
            'cyclomatic': cyclomatic,
            'halstead_volume': halstead['volume'],
            'halstead_difficulty': halstead['difficulty'],
            'halstead_effort': halstead['effort'],
            'operators': operators,
            'operands': operands,
            'maintainability': maintainability,
            'loc': float(loc)
        }
        if keep_label and label_column in df.columns:
            entry['label'] = row[label_column]
        rows.append(entry)
    return pd.DataFrame(rows) if rows else pd.DataFrame()

def analyze_vulnerability_detection(path):
    df = pd.read_csv(path)
    return analyze_dataframe(df, 'func', keep_label=True, label_column='label')

def analyze_code_search(path):
    df = pd.read_csv(path)
    return analyze_dataframe(df, 'code')

def analyze_code_translation(path):
    df = pd.read_csv(path)
    return analyze_dataframe(df, 'java')

def analyze_clone_detection(path):
    df = pd.read_csv(path)
    return analyze_dataframe(df, 'func1', keep_label=True, label_column='label')

def build_task_dataframe():
    paths = {
        'Vulnerability Detection': '/traincodex.csv',
        'Code Search': '/train (1).csv',
        'Code Translation': '/train.csv',
        'Clone Detection': 'train.csv'
    }
    frames = []
    for task, path in paths.items():
        analyzer = {
            'Vulnerability Detection': analyze_vulnerability_detection,
            'Code Search': analyze_code_search,
            'Code Translation': analyze_code_translation,
            'Clone Detection': analyze_clone_detection
        }[task]
        df = analyzer(path)
        if not df.empty:
            df['task'] = task
            frames.append(df)
    all_df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    metric_cols = ['ast_depth', 'cyclomatic', 'halstead_volume', 'halstead_difficulty',
                   'halstead_effort', 'operators', 'operands', 'maintainability', 'loc']
    for col in metric_cols:
        if col in all_df.columns:
            all_df[col] = pd.to_numeric(all_df[col], errors='coerce')
    return all_df

def create_summary_table(all_df):
    metrics = ['loc', 'ast_depth', 'cyclomatic', 'halstead_volume', 'halstead_difficulty', 'maintainability']
    stats = all_df.groupby('task')[metrics].agg(['mean', 'median', 'std']).round(2)
    stats.columns = ['_'.join(col) for col in stats.columns]
    stats.to_csv('dataset_metrics_summary.csv')
    print("\n=== Summary Statistics per Task ===")
    print(stats)

def create_kde_distributions(all_df):
    metrics = ['loc', 'cyclomatic', 'ast_depth', 'halstead_volume']
    log_metrics = ['loc', 'halstead_volume']
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    fig.suptitle('Kernel Density Estimates of Code Metrics across PEFT Tasks', fontsize=20, fontweight='bold')
    for ax, metric in zip(axes.flat, metrics):
        for i, task in enumerate(task_order):
            data = all_df[all_df['task'] == task][metric].dropna()
            if len(data) == 0:
                continue
            sns.kdeplot(data, ax=ax, fill=True, alpha=0.4, label=task, color=task_colors[i], log_scale=(metric in log_metrics))
            mean_val = data.mean()
            median_val = data.median()
            ax.axvline(mean_val, color=task_colors[i], linestyle='--', linewidth=1.5)
            ax.text(mean_val, ax.get_ylim()[1]*0.9, f'Mean: {mean_val:.1f}', color=task_colors[i], fontsize=10, ha='left')
            ax.axvline(median_val, color=task_colors[i], linestyle=':', linewidth=1.5)
            ax.text(median_val, ax.get_ylim()[1]*0.8, f'Median: {median_val:.1f}', color=task_colors[i], fontsize=10, ha='left')
        ax.set_xlabel(metric.replace('_', ' ').title(), fontsize=14)
        ax.set_ylabel('Density', fontsize=14)
        ax.legend(title='Task', fontsize=11)
        ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('metrics_kde_distributions.png', dpi=400, bbox_inches='tight')
    plt.close()

def create_violin_plots(all_df):
    metrics = ['cyclomatic', 'ast_depth', 'halstead_difficulty', 'maintainability']
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    fig.suptitle('Violin Plots with Mean/Median Annotations across PEFT Tasks', fontsize=20, fontweight='bold')
    for ax, metric in zip(axes.flat, metrics):
        sns.violinplot(data=all_df, x='task', y=metric, order=task_order, ax=ax, inner='quartile', cut=0)
        for i, task in enumerate(task_order):
            data = all_df[all_df['task'] == task][metric].dropna()
            if len(data) == 0:
                continue
            mean_val = data.mean()
            median_val = data.median()
            ax.text(i, mean_val, f'{mean_val:.1f}', ha='center', va='bottom', fontweight='bold', color='black', fontsize=10)
            ax.text(i, median_val, f'{median_val:.1f}', ha='center', va='top', color='white', fontweight='bold', fontsize=10)
        ax.set_xlabel('')
        ax.set_ylabel(metric.replace('_', ' ').title(), fontsize=14)
        ax.tick_params(axis='x', rotation=12)
        ax.grid(True, axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig('metrics_violin_plots.png', dpi=400, bbox_inches='tight')
    plt.close()

def create_regression_scatters(all_df):
    pairs = [('loc', 'cyclomatic'), ('loc', 'halstead_volume'), ('cyclomatic', 'ast_depth'), ('halstead_difficulty', 'maintainability')]
    fig, axes = plt.subplots(2, 2, figsize=(18, 14))
    fig.suptitle('Key Metric Correlations with Regression Lines and Stats per Task', fontsize=20, fontweight='bold')
    for ax, (x, y) in zip(axes.flat, pairs):
        subset = all_df[[x, y, 'task']].dropna()
        if len(subset) > 15000:
            subset = subset.sample(15000, random_state=42)
        sns.scatterplot(data=subset, x=x, y=y, hue='task', hue_order=task_order, alpha=0.5, s=30, ax=ax, edgecolor=None)
        sns.regplot(data=subset, x=x, y=y, scatter=False, ax=ax, ci=95, truncate=False, color='black', line_kws={'linewidth':1.5})
        y_pos = 0.95
        for task in task_order:
            task_data = subset[subset['task'] == task]
            if len(task_data) < 10:
                continue
            x_vals = task_data[x]
            y_vals = task_data[y]
            if x_vals.nunique() <= 1 or y_vals.nunique() <= 1:
                ax.text(0.05, y_pos, f'{task}: Insufficient variation for regression', 
                        transform=ax.transAxes, fontsize=10, bbox=dict(facecolor='white', alpha=0.8))
            else:
                slope, intercept, r, p, se = stats.linregress(x_vals, y_vals)
                r2 = r**2
                ax.text(0.05, y_pos, f'{task}: Slope={slope:.2f}, R²={r2:.3f}, p={p:.2e}',
                        transform=ax.transAxes, fontsize=10, bbox=dict(facecolor='white', alpha=0.8), 
                        color=task_colors[task_order.index(task)])
            y_pos -= 0.08
        ax.set_xlabel(x.replace('_', ' ').title(), fontsize=14)
        ax.set_ylabel(y.replace('_', ' ').title(), fontsize=14)
        if x in ['loc', 'halstead_volume'] or y in ['loc', 'halstead_volume']:
            if x in ['loc', 'halstead_volume']:
                ax.set_xscale('log')
            if y in ['loc', 'halstead_volume']:
                ax.set_yscale('log')
        ax.grid(True, alpha=0.3)
        ax.legend().remove()
    plt.tight_layout()
    plt.savefig('metrics_regression_scatters.png', dpi=400, bbox_inches='tight')
    plt.close()

def create_per_task_correlation_heatmaps(all_df):
    metrics = ['loc', 'ast_depth', 'cyclomatic', 'halstead_volume', 'halstead_difficulty', 'operators', 'operands', 'maintainability']
    tasks = all_df['task'].unique()
    n_tasks = len(tasks)
    cols = 2
    rows = (n_tasks + 1) // 2
    fig, axes = plt.subplots(rows, cols, figsize=(12 * cols, 10 * rows))
    fig.suptitle('Spearman Correlation Matrix of Code Complexity Metrics per Task', fontsize=24, fontweight='bold', y=0.98)
    if n_tasks == 1:
        axes = np.array([[axes]])
    elif rows == 1:
        axes = np.array([axes]) if cols == 1 else axes.reshape(1, -1)
    axes = axes.flatten()
    for idx, task in enumerate(tasks):
        ax = axes[idx]
        task_df = all_df[all_df['task'] == task][metrics].dropna()
        if len(task_df) < 2:
            ax.text(0.5, 0.5, 'Insufficient data for correlation', ha='center', va='center', fontsize=16, transform=ax.transAxes)
        else:
            corr = task_df.corr(method='spearman').round(2)
            sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, square=True, linewidths=0.5,
                        ax=ax, cbar_kws={"shrink": 0.8}, fmt='.2f')
        ax.set_title(task, fontsize=18, fontweight='bold', pad=20)
    for j in range(idx + 1, len(axes)):
        fig.delaxes(axes[j])
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig('metrics_per_task_correlation_heatmaps.png', dpi=400, bbox_inches='tight')
    plt.close()

def create_radar_chart(all_df):
    metrics = ['loc', 'ast_depth', 'cyclomatic', 'halstead_volume', 'halstead_difficulty', 'maintainability']
    means = all_df.groupby('task')[metrics].mean()
    means_norm = (means - means.min()) / (means.max() - means.min() + 1e-8)
    categories = [m.replace('_', ' ').title() for m in metrics]
    N = len(categories)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]
    fig, ax = plt.subplots(figsize=(11, 11), subplot_kw=dict(polar=True))
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)
    plt.xticks(angles[:-1], categories, size=13)
    plt.ylim(0, 1)
    plt.yticks([0.2, 0.4, 0.6, 0.8], ["0.2", "0.4", "0.6", "0.8"], size=10)
    for i, task in enumerate(task_order):
        if task not in means_norm.index:
            continue
        values = means_norm.loc[task].tolist() + [means_norm.loc[task].iloc[0]]
        ax.plot(angles, values, linewidth=3.5, linestyle='solid', label=task, color=task_colors[i])
        ax.fill(angles, values, alpha=0.3, color=task_colors[i])
        for j in range(N):
            val = means.loc[task, metrics[j]]
            ax.text(angles[j], values[j] + 0.05, f'{val:.1f}', ha='center', va='center', fontsize=9, color=task_colors[i], fontweight='bold')
    ax.set_title('Normalized Average Complexity Profile across PEFT Tasks\n(with raw mean values annotated)', size=20, fontweight='bold', pad=40)
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0), fontsize=12)
    plt.savefig('metrics_radar_chart.png', dpi=400, bbox_inches='tight')
    plt.close()

def create_ecdf_plots(all_df):
    metrics = ['loc', 'cyclomatic', 'ast_depth', 'halstead_volume']
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    fig.suptitle('Empirical Cumulative Distribution Functions with Median Annotations', fontsize=20, fontweight='bold')
    for ax, metric in zip(axes.flat, metrics):
        for i, task in enumerate(task_order):
            data = all_df[all_df['task'] == task][metric].dropna().sort_values()
            if len(data) == 0:
                continue
            sns.ecdfplot(data=data, ax=ax, label=task, color=task_colors[i], log_scale=(metric in ['loc', 'halstead_volume']))
            median_val = data.median()
            ax.axvline(median_val, color=task_colors[i], linestyle='--', alpha=0.7)
            ax.text(median_val, 0.5, f'{task} Median: {median_val:.1f}', rotation=90, va='center', color=task_colors[i], fontsize=10, bbox=dict(facecolor='white', alpha=0.7))
        ax.set_xlabel(metric.replace('_', ' ').title(), fontsize=14)
        ax.set_ylabel('Proportion', fontsize=14)
        ax.legend(title='Task', fontsize=11)
        ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('metrics_ecdf.png', dpi=400, bbox_inches='tight')
    plt.close()

def main():
    print('Generating enhanced research-grade visualizations with detailed annotations...')
    all_df = build_task_dataframe()
    if all_df.empty:
        print('No data loaded. Check paths/columns.')
        return
    print(f'Total valid code samples: {len(all_df)}')
    create_summary_table(all_df)
    create_kde_distributions(all_df)
    print('→ metrics_kde_distributions.png (annotated means/medians)')
    create_violin_plots(all_df)
    print('→ metrics_violin_plots.png (annotated means/medians)')
    create_regression_scatters(all_df)
    print('→ metrics_regression_scatters.png (per-task regression stats - fixed)')
    create_per_task_correlation_heatmaps(all_df)
    print('→ metrics_per_task_correlation_heatmaps.png (separate heatmap per task)')
    create_radar_chart(all_df)
    print('→ metrics_radar_chart.png (annotated raw means)')
    create_ecdf_plots(all_df)
    print('→ metrics_ecdf.png (annotated medians)')


if __name__ == '__main__':
    main()

# Ablation Study

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import time
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, cohen_kappa_score, jaccard_score
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

class CodeDataset(Dataset):
    def __init__(self, csv_path, tokenizer, max_len=512):
        df = pd.read_csv(csv_path)
        self.codes = df['code'].tolist()
        self.labels = df['label'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.codes)
    
    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.codes[idx], 
            truncation=True, 
            padding='max_length', 
            max_length=self.max_len, 
            return_tensors='pt'
        )
        return (
            enc['input_ids'].squeeze(0), 
            enc['attention_mask'].squeeze(0), 
            torch.tensor(self.labels[idx], dtype=torch.long)
        )

class AttentionAlignedAdapter(nn.Module):
    def __init__(self, hidden_dim=768, rank=64, pooling='mean', fusion='add', scaled=False):
        super().__init__()
        self.Wd = nn.Linear(hidden_dim, rank, bias=False)
        self.Wu = nn.Linear(rank, hidden_dim, bias=False)
        self.pooling = pooling
        self.fusion = fusion
        self.scaled = scaled
        if scaled:
            self.alpha = nn.Parameter(torch.ones(1))
        if fusion == 'concat':
            self.fusion_proj = nn.Linear(hidden_dim * 2, hidden_dim, bias=False)
        
    def forward(self, X, S, A):
        if self.pooling == 'mean':
            S_pooled = S.mean(dim=1)
        elif self.pooling == 'max':
            S_pooled = S.max(dim=1)[0]
        
        Z = self.Wd(X)
        Zp = torch.matmul(S_pooled, Z)
        deltaA = self.Wu(Zp)
        
        if self.scaled:
            deltaA = self.alpha * deltaA
        
        if self.fusion == 'add':
            return A + deltaA
        elif self.fusion == 'concat':
            return self.fusion_proj(torch.cat([A, deltaA], dim=-1))
        
        return A + deltaA

class PEFTSelfAttention(nn.Module):
    def __init__(self, attn_self, attn_output, config):
        super().__init__()
        self.config = config
        self.self_attn = attn_self
        self.out_dense = attn_output.dense
        self.out_bias = attn_output.dense.bias
        
        if config['adapter']:
            self.adapter = AttentionAlignedAdapter(
                hidden_dim=768, 
                rank=config['rank'],
                pooling=config.get('pooling', 'mean'),
                fusion=config.get('fusion', 'add'),
                scaled=config.get('scaled', False)
            )
        else:
            self.adapter = None
        
        self.out_dense.weight.requires_grad = False
        self.out_bias.requires_grad = config['bias']
        
        if config.get('trainable_attn', False):
            for p in self.self_attn.parameters():
                p.requires_grad = True
        else:
            for p in self.self_attn.parameters():
                p.requires_grad = False
    
    def forward(self, hidden_states, attention_mask):
        B, T, H = hidden_states.size()
        q = self.self_attn.query(hidden_states)
        k = self.self_attn.key(hidden_states)
        v = self.self_attn.value(hidden_states)
        
        q = q.view(B, T, self.self_attn.num_attention_heads, -1).transpose(1, 2)
        k = k.view(B, T, self.self_attn.num_attention_heads, -1).transpose(1, 2)
        v = v.view(B, T, self.self_attn.num_attention_heads, -1).transpose(1, 2)
        
        scores = torch.matmul(q, k.transpose(-2, -1)) / np.sqrt(q.size(-1))
        scores = scores + attention_mask
        S = F.softmax(scores, dim=-1)
        A = torch.matmul(S, v)
        A = A.transpose(1, 2).contiguous().view(B, T, H)
        
        if self.adapter is not None:
            A = self.adapter(hidden_states, S, A)
        
        entropy = (-S * torch.log(S + 1e-9)).sum(dim=-1).mean()
        return self.out_dense(A), entropy

class PEFTEncoderLayer(nn.Module):
    def __init__(self, layer, config, layer_idx, apply_peft):
        super().__init__()
        self.apply_peft = apply_peft
        
        if apply_peft:
            self.attn = PEFTSelfAttention(layer.attention.self, layer.attention.output, config)
        else:
            self.attn = PEFTSelfAttention(
                layer.attention.self, 
                layer.attention.output, 
                {'adapter': False, 'bias': False, 'rank': 64, 'trainable_attn': False}
            )
        
        self.attn_ln = layer.attention.output.LayerNorm
        self.ffn_int = layer.intermediate.dense
        self.ffn_out = layer.output.dense
        self.ffn_ln = layer.output.LayerNorm
        
        for p in self.attn_ln.parameters():
            p.requires_grad = False
        for p in self.ffn_int.parameters():
            p.requires_grad = False
        for p in self.ffn_out.parameters():
            p.requires_grad = False
        for p in self.ffn_ln.parameters():
            p.requires_grad = False
    
    def forward(self, x, attn_mask):
        attn_out, entropy = self.attn(x, attn_mask)
        x = self.attn_ln(x + attn_out)
        ff = self.ffn_out(F.gelu(self.ffn_int(x)))
        x = self.ffn_ln(x + ff)
        return x, entropy

class PEFTUniXcoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.model = AutoModel.from_pretrained("microsoft/unixcoder-base")
        
        for p in self.model.parameters():
            p.requires_grad = False
        
        self.embeddings = self.model.embeddings
        self.layers = nn.ModuleList()
        
        num_layers = len(self.model.encoder.layer)
        layer_scope = config.get('layer_scope', 'all')
        
        for i, l in enumerate(self.model.encoder.layer):
            apply_peft = False
            
            if layer_scope == 'all':
                apply_peft = True
            elif layer_scope == 'middle':
                apply_peft = (i >= 4 and i <= 7)
            elif layer_scope == 'specific':
                apply_peft = (i < config.get('num_layers', num_layers))
            
            self.layers.append(PEFTEncoderLayer(l, config, i, apply_peft))
        
        self.classifier = nn.Linear(768, 2)
    
    def forward(self, input_ids, attention_mask):
        attn_mask = (1.0 - attention_mask.unsqueeze(1).unsqueeze(2)) * -10000.0
        x = self.embeddings(input_ids)
        entropies = []
        
        for layer in self.layers:
            x, ent = layer(x, attn_mask)
            entropies.append(ent)
        
        logits = self.classifier(x[:, 0, :])
        return logits, torch.stack(entropies).mean()

def compute_metrics(y_true, y_pred, avg="binary"):
    return {
        "acc": accuracy_score(y_true, y_pred),
        "bal_acc": balanced_accuracy_score(y_true, y_pred),
        "prec": precision_score(y_true, y_pred, average=avg, zero_division=0),
        "rec": recall_score(y_true, y_pred, average=avg, zero_division=0),
        "f1": f1_score(y_true, y_pred, average=avg, zero_division=0),
        "jacc": jaccard_score(y_true, y_pred, average=avg, zero_division=0),
        "mcc": matthews_corrcoef(y_true, y_pred),
        "kappa": cohen_kappa_score(y_true, y_pred)
    }

def train_and_evaluate(config, train_loader, val_loader, test_loader, device):
    model = PEFTUniXcoder(config).to(device)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    
    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()), 
        lr=2e-4
    )
    
    results = {
        'config': config,
        'trainable_params': trainable_params,
        'total_params': total_params,
        'trainable_ratio': trainable_params / total_params * 100,
        'epochs': []
    }
    
    total_train_time = 0
    
    for epoch in range(5):
        model.train()
        train_loss, y_true, y_pred = 0, [], []
        epoch_start = time.time()
        
        for ids, mask, labels in train_loader:
            ids, mask, labels = ids.to(device), mask.to(device), labels.to(device)
            
            logits, _ = model(ids, mask)
            loss = F.cross_entropy(logits, labels)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            train_loss += loss.item()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(torch.argmax(logits, 1).cpu().numpy())
        
        epoch_time = time.time() - epoch_start
        total_train_time += epoch_time
        train_metrics = compute_metrics(y_true, y_pred)
        
        model.eval()
        val_loss, y_true, y_pred = 0, [], []
        
        with torch.no_grad():
            for ids, mask, labels in val_loader:
                ids, mask, labels = ids.to(device), mask.to(device), labels.to(device)
                logits, _ = model(ids, mask)
                val_loss += F.cross_entropy(logits, labels).item()
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(torch.argmax(logits, 1).cpu().numpy())
        
        val_metrics = compute_metrics(y_true, y_pred)
        
        results['epochs'].append({
            'epoch': epoch + 1,
            'train_loss': train_loss / len(train_loader),
            'train_f1': train_metrics['f1'],
            'val_loss': val_loss / len(val_loader),
            'val_f1': val_metrics['f1'],
            'epoch_time': epoch_time
        })
    
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
    
    model.eval()
    y_true, y_pred = [], []
    infer_start = time.time()
    
    with torch.no_grad():
        for ids, mask, labels in test_loader:
            ids, mask = ids.to(device), mask.to(device)
            logits, _ = model(ids, mask)
            y_pred.extend(torch.argmax(logits, 1).cpu().numpy())
            y_true.extend(labels.numpy())
    
    infer_time = time.time() - infer_start
    peak_memory = torch.cuda.max_memory_allocated() / 1024**2 if torch.cuda.is_available() else 0
    
    test_metrics = compute_metrics(y_true, y_pred, avg="macro")
    
    results['test_metrics'] = test_metrics
    results['total_train_time'] = total_train_time
    results['inference_time'] = infer_time
    results['peak_memory_mb'] = peak_memory
    results['throughput'] = len(test_loader.dataset) / infer_time
    
    return results

def main():
    tokenizer = AutoTokenizer.from_pretrained("microsoft/unixcoder-base")
    
    full_train = CodeDataset('/traincodex.csv', tokenizer)
    test_dataset = CodeDataset('/testcodex.csv', tokenizer)
    
    val_size = int(0.1 * len(full_train))
    train_size = len(full_train) - val_size
    train_dataset, val_dataset = random_split(full_train, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=8, num_workers=4)
    
    ablation_configs = [
        {
            'name': 'baseline_frozen',
            'adapter': False,
            'bias': False,
            'rank': 64,
            'layer_scope': 'all',
            'trainable_attn': False
        },
        {
            'name': 'bias_only',
            'adapter': False,
            'bias': True,
            'rank': 64,
            'layer_scope': 'all',
            'trainable_attn': False
        },
        {
            'name': 'trainable_attention',
            'adapter': False,
            'bias': False,
            'rank': 64,
            'layer_scope': 'all',
            'trainable_attn': True
        },
        {
            'name': 'full_method_mean',
            'adapter': True,
            'bias': True,
            'rank': 64,
            'layer_scope': 'all',
            'trainable_attn': False,
            'pooling': 'mean',
            'fusion': 'add',
            'scaled': False
        },
        {
            'name': 'full_method_max',
            'adapter': True,
            'bias': True,
            'rank': 64,
            'layer_scope': 'all',
            'trainable_attn': False,
            'pooling': 'max',
            'fusion': 'add',
            'scaled': False
        },
        {
            'name': 'fusion_concat',
            'adapter': True,
            'bias': True,
            'rank': 64,
            'layer_scope': 'all',
            'trainable_attn': False,
            'pooling': 'mean',
            'fusion': 'concat',
            'scaled': False
        },
        {
            'name': 'scaled_fusion',
            'adapter': True,
            'bias': True,
            'rank': 64,
            'layer_scope': 'all',
            'trainable_attn': False,
            'pooling': 'mean',
            'fusion': 'add',
            'scaled': True
        },
        {
            'name': 'middle_layers_only',
            'adapter': True,
            'bias': True,
            'rank': 64,
            'layer_scope': 'middle',
            'trainable_attn': False,
            'pooling': 'mean',
            'fusion': 'add',
            'scaled': False
        },
        {
            'name': 'first_1_layer',
            'adapter': True,
            'bias': True,
            'rank': 64,
            'layer_scope': 'specific',
            'num_layers': 1,
            'trainable_attn': False,
            'pooling': 'mean',
            'fusion': 'add',
            'scaled': False
        },
        {
            'name': 'first_3_layers',
            'adapter': True,
            'bias': True,
            'rank': 64,
            'layer_scope': 'specific',
            'num_layers': 3,
            'trainable_attn': False,
            'pooling': 'mean',
            'fusion': 'add',
            'scaled': False
        },
        {
            'name': 'first_6_layers',
            'adapter': True,
            'bias': True,
            'rank': 64,
            'layer_scope': 'specific',
            'num_layers': 6,
            'trainable_attn': False,
            'pooling': 'mean',
            'fusion': 'add',
            'scaled': False
        },
    ]
    
    all_results = []
    
    for i, config in enumerate(ablation_configs):
        print("\n" + "=" * 100)
        print(f"ABLATION {i+1}/{len(ablation_configs)}: {config['name']}")
        print("=" * 100)
        
        try:
            results = train_and_evaluate(config, train_loader, val_loader, test_loader, device)
            all_results.append(results)
            
            print(f"\nTrainable Parameters: {results['trainable_params']:,} ({results['trainable_ratio']:.4f}%)")
            print(f"Final Val F1: {results['epochs'][-1]['val_f1']:.4f}")
            print(f"Test F1: {results['test_metrics']['f1']:.4f}")
            print(f"Total Training Time: {results['total_train_time']:.2f}s")
            print(f"Inference Time: {results['inference_time']:.2f}s")
            print(f"Peak Memory: {results['peak_memory_mb']:.2f}MB")
            print(f"Throughput: {results['throughput']:.2f} samples/sec")
            
        except Exception as e:
            print(f"ERROR: {str(e)}")
            continue
    
    with open('ablation_results.json', 'w') as f:
        json.dump(all_results, f, indent=2)
    
    print("\n" + "=" * 100)
    print("ABLATION STUDY SUMMARY")
    print("=" * 100)
    
    for res in all_results:
        print(f"\n{res['config']['name']}:")
        print(f"  Trainable: {res['trainable_params']:,} ({res['trainable_ratio']:.4f}%)")
        print(f"  Test F1: {res['test_metrics']['f1']:.4f}")
        print(f"  Train Time: {res['total_train_time']:.2f}s")
        print(f"  Inference: {res['inference_time']:.2f}s")
        print(f"  Memory: {res['peak_memory_mb']:.2f}MB")

if __name__ == "__main__":
    main()

# A-LoRE train-test, loss, F1 loss 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import time
import copy
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CodeDataset(Dataset):
    def __init__(self, csv_path, tokenizer, max_len=512):
        df = pd.read_csv(csv_path)
        self.codes = df['code'].tolist()
        self.labels = df['label'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.codes)
    def __getitem__(self, idx):
        enc = self.tokenizer(self.codes[idx], truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
        return enc['input_ids'].squeeze(0), enc['attention_mask'].squeeze(0), torch.tensor(self.labels[idx], dtype=torch.long)

class AttentionAlignedAdapter(nn.Module):
    def __init__(self, hidden_dim=768, rank=64):
        super().__init__()
        self.Wd = nn.Linear(hidden_dim, rank, bias=False)
        self.Wu = nn.Linear(rank, hidden_dim, bias=False)
    def forward(self, X, S):
        Z = self.Wd(X)
        Zp = torch.matmul(S, Z)
        return self.Wu(Zp)

class PEFTSelfAttention(nn.Module):
    def __init__(self, attn_self, attn_output, hidden_dim=768, rank=64):
        super().__init__()
        self.self_attn = attn_self
        self.Wo = attn_output.dense
        self.bo = attn_output.dense.bias
        self.adapter = AttentionAlignedAdapter(hidden_dim, rank)
        
        self.Wo.weight.requires_grad = False
        self.bo.requires_grad = True
        self.stored_S = None
        
    def forward(self, hidden_states, attention_mask):
        B,T,H = hidden_states.size()
        q = self.self_attn.query(hidden_states)
        k = self.self_attn.key(hidden_states)
        v = self.self_attn.value(hidden_states)
        
        q = q.view(B,T,self.self_attn.num_attention_heads,-1).transpose(1,2)
        k = k.view(B,T,self.self_attn.num_attention_heads,-1).transpose(1,2)
        v = v.view(B,T,self.self_attn.num_attention_heads,-1).transpose(1,2)
        
        scores = torch.matmul(q, k.transpose(-2,-1)) / np.sqrt(q.size(-1))
        scores = scores + attention_mask
        S = F.softmax(scores, dim=-1)
        
        if not S.requires_grad:
            S.requires_grad_(True)
        S.retain_grad() 
        self.stored_S = S 
        
        A = torch.matmul(S, v)
        A = A.transpose(1,2).contiguous().view(B,T,H)
        
        deltaA = self.adapter(hidden_states, S.mean(dim=1))
        A = A + deltaA
        
        entropy = (-S * torch.log(S + 1e-9)).sum(dim=-1).mean()
        return self.Wo(A), entropy

class PEFTEncoderLayer(nn.Module):
    def __init__(self, layer):
        super().__init__()
        self.attn = PEFTSelfAttention(layer.attention.self, layer.attention.output)
        self.attn_ln = layer.attention.output.LayerNorm
        self.ffn_int = layer.intermediate.dense
        self.ffn_out = layer.output.dense
        self.ffn_ln = layer.output.LayerNorm
        
        for p in self.attn_ln.parameters(): p.requires_grad = False
        for p in self.ffn_int.parameters(): p.requires_grad = False
        for p in self.ffn_out.parameters(): p.requires_grad = False
        for p in self.ffn_ln.parameters(): p.requires_grad = False

    def forward(self, x, attn_mask):
        attn_out, entropy = self.attn(x, attn_mask)
        x = self.attn_ln(x + attn_out)
        ff = self.ffn_out(F.gelu(self.ffn_int(x)))
        x = self.ffn_ln(x + ff)
        return x, entropy

class PEFTUniXcoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = AutoModel.from_pretrained("microsoft/unixcoder-base")
        for p in self.model.parameters():
            p.requires_grad = False
        
        self.layers = nn.ModuleList([PEFTEncoderLayer(l) for l in self.model.encoder.layer])
        self.embeddings = self.model.embeddings
        self.classifier = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        attn_mask = (1.0 - attention_mask.unsqueeze(1).unsqueeze(2)) * -10000.0
        x = self.embeddings(input_ids)
        entropies = []
        for layer in self.layers:
            x, ent = layer(x, attn_mask)
            entropies.append(ent)
        logits = self.classifier(x[:,0,:])
        return logits, torch.stack(entropies).mean()

def compute_metrics(y_true, y_pred, avg="binary"):
    return {
        "f1": f1_score(y_true, y_pred, average=avg, zero_division=0),
        "acc": accuracy_score(y_true, y_pred)
    }

tokenizer = AutoTokenizer.from_pretrained("microsoft/unixcoder-base")
full_train = CodeDataset('/traincodex.csv', tokenizer)
test_dataset = CodeDataset('/testcodex.csv', tokenizer)

val_size = int(0.1 * len(full_train))
train_size = len(full_train) - val_size
train_dataset, val_dataset = random_split(full_train, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)
test_loader = DataLoader(test_dataset, batch_size=4)

model = PEFTUniXcoder().to(device)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-4)

total_steps = len(train_loader) * 10
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1*total_steps), num_training_steps=total_steps)

best_val_f1 = 0.0
best_model_state = None
history = {'train_loss': [], 'val_loss': [], 'train_f1': [], 'val_f1': []}

print("TRAINABLE PARAMETERS:")
for name, p in model.named_parameters():
    if p.requires_grad:
        print(f"✓ {name}: {p.numel():,}")

for epoch in range(10):
    model.train()
    train_loss, y_true, y_pred = 0, [], []
    
    for ids, mask, labels in train_loader:
        ids, mask, labels = ids.to(device), mask.to(device), labels.to(device)
        logits, _ = model(ids, mask)
        loss = F.cross_entropy(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        train_loss += loss.item()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(torch.argmax(logits, 1).cpu().numpy())
        
    epoch_train_loss = train_loss/len(train_loader)
    epoch_train_f1 = f1_score(y_true, y_pred, average='binary', zero_division=0)

    model.eval()
    val_loss, y_true, y_pred = 0, [], []
    with torch.no_grad():
        for ids, mask, labels in val_loader:
            ids, mask, labels = ids.to(device), mask.to(device), labels.to(device)
            logits, _ = model(ids, mask)
            val_loss += F.cross_entropy(logits, labels).item()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(torch.argmax(logits, 1).cpu().numpy())
            
    epoch_val_loss = val_loss/len(val_loader)
    epoch_val_f1 = f1_score(y_true, y_pred, average='binary', zero_division=0)
    
    history['train_loss'].append(epoch_train_loss)
    history['val_loss'].append(epoch_val_loss)
    history['train_f1'].append(epoch_train_f1)
    history['val_f1'].append(epoch_val_f1)

    print(f"EPOCH {epoch+1}: Train Loss {epoch_train_loss:.4f} | Val F1: {epoch_val_f1:.4f}")

    if epoch_val_f1 > best_val_f1:
        best_val_f1 = epoch_val_f1
        best_model_state = copy.deepcopy(model.state_dict())

model.load_state_dict(best_model_state)

model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for ids, mask, labels in test_loader:
        ids, mask = ids.to(device), mask.to(device)
        logits, _ = model(ids, mask)
        y_pred.extend(torch.argmax(logits, 1).cpu().numpy())
        y_true.extend(labels.numpy())

print(f"\nFINAL TEST F1 (Best Epoch): {f1_score(y_true, y_pred, average='macro'):.4f}")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['train_f1'], label='Train F1')
plt.plot(history['val_f1'], label='Val F1')
plt.title('F1 Score Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()
plt.savefig('performance_graphs.png')
print("Graphs saved to performance_graphs.png")

print("\nSTATISTICAL PROOF (GRADIENT NORM)")
model.train()
subset_loader = DataLoader(Subset(test_dataset, np.random.choice(len(test_dataset), 50, replace=False)), batch_size=1)
grad_norms = []

for idx, (ids, mask, labels) in enumerate(subset_loader):
    ids, mask, labels = ids.to(device), mask.to(device), labels.to(device)
    optimizer.zero_grad()
    logits, _ = model(ids, mask)
    loss = F.cross_entropy(logits, labels)
    loss.backward()
    
    S = model.layers[-1].attn.stored_S
    if S.grad is not None:
        grad_norms.append(torch.norm(S.grad).item())

avg_norm = np.mean(grad_norms)
print(f"Avg Gradient Norm on S: {avg_norm:.6f}")
if avg_norm > 0:
    print("SUCCESS.")

# otivational analysis of A-LoRE

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel

class AttentionAlignedAdapter(nn.Module):
    def __init__(self, hidden_dim=768, rank=64):
        super().__init__()
        self.Wd = nn.Linear(hidden_dim, rank, bias=False)
        self.Wu = nn.Linear(rank, hidden_dim, bias=False)
        
    def forward(self, X, S):
        Z = self.Wd(X)
        Zp = torch.matmul(S, Z) 
        return self.Wu(Zp)

class PEFTSelfAttention(nn.Module):
    def __init__(self, attn_self, attn_output, hidden_dim=768, rank=64):
        super().__init__()
        self.self_attn = attn_self
        self.out_dense = attn_output.dense
        self.out_bias = attn_output.dense.bias
        self.adapter = AttentionAlignedAdapter(hidden_dim, rank)
        
        self.out_dense.weight.requires_grad = False
        self.out_bias.requires_grad = True
        self.adapter_enabled = True 

    def forward(self, hidden_states, attention_mask):
        B,T,H = hidden_states.size()
        q = self.self_attn.query(hidden_states)
        k = self.self_attn.key(hidden_states)
        v = self.self_attn.value(hidden_states)
        
        q = q.view(B,T,self.self_attn.num_attention_heads,-1).transpose(1,2)
        k = k.view(B,T,self.self_attn.num_attention_heads,-1).transpose(1,2)
        v = v.view(B,T,self.self_attn.num_attention_heads,-1).transpose(1,2)
        
        scores = torch.matmul(q, k.transpose(-2,-1)) / np.sqrt(q.size(-1))
        scores = scores + attention_mask
        S = F.softmax(scores, dim=-1)
        
        A = torch.matmul(S, v)
        A = A.transpose(1,2).contiguous().view(B,T,H)
        
        if self.adapter_enabled:
            deltaA = self.adapter(hidden_states, S.mean(dim=1))
            A = A + deltaA
            
        return self.out_dense(A)

class PEFTEncoderLayer(nn.Module):
    def __init__(self, layer):
        super().__init__()
        self.attn = PEFTSelfAttention(layer.attention.self, layer.attention.output)
        self.attn_ln = layer.attention.output.LayerNorm
        self.ffn_int = layer.intermediate.dense
        self.ffn_out = layer.output.dense
        self.ffn_ln = layer.output.LayerNorm
        
        for p in self.attn_ln.parameters(): p.requires_grad = False
        for p in self.ffn_int.parameters(): p.requires_grad = False
        for p in self.ffn_out.parameters(): p.requires_grad = False
        for p in self.ffn_ln.parameters(): p.requires_grad = False

    def forward(self, x, attn_mask):
        attn_out = self.attn(x, attn_mask)
        x = self.attn_ln(x + attn_out)
        ff = self.ffn_out(F.gelu(self.ffn_int(x)))
        x = self.ffn_ln(x + ff)
        return x

class PEFTUniXcoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = AutoModel.from_pretrained("microsoft/unixcoder-base")
        for p in self.model.parameters():
            p.requires_grad = False
            
        self.layers = nn.ModuleList([PEFTEncoderLayer(l) for l in self.model.encoder.layer])
        self.embeddings = self.model.embeddings
        self.classifier = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        attn_mask = (1.0 - attention_mask.unsqueeze(1).unsqueeze(2)) * -10000.0
        x = self.embeddings(input_ids)
        x.requires_grad_(True)
        x.retain_grad()
        self.input_embeddings = x 
        
        for layer in self.layers:
            x = layer(x, attn_mask)
        logits = self.classifier(x[:,0,:])
        return logits

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("microsoft/unixcoder-base")

code_sample = """
static int subframe_count_exact(FlacEncodeContext *s, FlacSubframe *sub, int pred_order)
{
    int p, porder, psize;
    int i, part_end;
    int count = 0;
    count += 8;
    if (sub->type == FLAC_SUBFRAME_CONSTANT) {
        count += sub->obits;
    } else {
        count += pred_order * sub->obits;
        porder = sub->rc.porder;
        psize  = s->frame.blocksize >> porder;
        for (p = 0; p < 1 << porder; p++) {
            int k = sub->rc.params[p];
            count += rice_count_exact(&sub->residual[i], part_end - i, k);
            i = part_end;
        }
    }
    return count;
}
"""

inputs = tokenizer(code_sample, return_tensors='pt', truncation=True, max_length=512)
ids = inputs['input_ids'].to(device)
mask = inputs['attention_mask'].to(device)
label = torch.tensor([1], device=device)

def get_saliency_map(model, ids, mask, label, use_adapter=True):
    for layer in model.layers:
        layer.attn.adapter_enabled = use_adapter
        
    model.zero_grad()
    logits = model(ids, mask)
    loss = F.cross_entropy(logits, label)
    loss.backward()
    
    grads = model.input_embeddings.grad
    if grads is None:
        return np.zeros(ids.shape[1])
        
    saliency = torch.abs(grads).sum(dim=2).squeeze(0)
    saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-9)
    return saliency.detach().cpu().numpy()

def micro_train_simulation(model, ids, mask, label, steps=20, use_adapter=True):
    trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
    if not trainable_params:
        return

    optimizer = torch.optim.AdamW(trainable_params, lr=1e-3)
    model.train()
    
    for layer in model.layers:
        layer.attn.adapter_enabled = use_adapter
        
    for _ in range(steps):
        optimizer.zero_grad()
        logits = model(ids, mask)
        loss = F.cross_entropy(logits, label)
        loss.backward()
        optimizer.step()

model = PEFTUniXcoder().to(device)

saliency_frozen = get_saliency_map(model, ids, mask, label, use_adapter=False)

micro_train_simulation(model, ids, mask, label, steps=30, use_adapter=True)
saliency_peft = get_saliency_map(model, ids, mask, label, use_adapter=True)

tokens = tokenizer.convert_ids_to_tokens(ids[0])
clean_tokens = [t.replace('Ġ', '') for t in tokens]

start_idx = 0
for i, t in enumerate(clean_tokens):
    if 'for' in t:
        start_idx = max(0, i - 10)
        break
end_idx = min(len(clean_tokens), start_idx + 80)

x_range = np.arange(start_idx, end_idx)
y_frozen = saliency_frozen[start_idx:end_idx]
y_peft = saliency_peft[start_idx:end_idx]

plt.figure(figsize=(16, 8))
plt.style.use('seaborn-v0_8-whitegrid')

plt.fill_between(x_range, y_frozen, color='gray', alpha=0.3, label='Frozen Model (Baseline Limit)')
plt.plot(x_range, y_frozen, color='gray', linestyle='--', linewidth=1.5)

plt.fill_between(x_range, y_peft, color='crimson', alpha=0.4, label='PEFT (Our Mitigation)')
plt.plot(x_range, y_peft, color='crimson', linewidth=2.5)

plt.title("Figure 4: Impact Analysis - Limits vs. Mitigation", fontsize=16, fontweight='bold')
plt.xlabel("Code Tokens", fontsize=14)
plt.ylabel("Saliency / Feature Importance", fontsize=14)
plt.xticks(x_range, clean_tokens[start_idx:end_idx], rotation=90, fontsize=10)
plt.legend(fontsize=12, loc='upper left', frameon=True)

plt.tight_layout()
plt.savefig("peft_impact_analysis.png", dpi=300)
print("Observation generated: peft_impact_analysis.png")